In [11]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

class PortfolioOptimizer:
    def __init__(self, returns_data):
        self.returns = returns_data
        self.n_assets = len(returns_data.columns)
        self.mean_returns = returns_data.mean()
        self.cov_matrix = returns_data.cov()
        
    def calculate_portfolio_metrics(self, weights):
        annual_return = np.sum(self.mean_returns * weights) * 252
        annual_volatility = np.sqrt(np.dot(weights.T, np.dot(self.cov_matrix * 252, weights)))
        sharpe_ratio = annual_return / annual_volatility
        
        return {
            'return': annual_return,
            'risk': annual_volatility,
            'sharpe_ratio': sharpe_ratio
        }
    
    def optimize_portfolio(self):
        constraints = [{'type': 'eq', 'fun': lambda x: np.sum(x) - 1}] 
        bounds = tuple((0, 1) for _ in range(self.n_assets)) 

        initial_weights = np.array([1/self.n_assets] * self.n_assets)
        
        result = minimize(
            lambda w: -self.calculate_portfolio_metrics(w)['sharpe_ratio'],
            initial_weights,
            method='SLSQP',
            bounds=bounds,
            constraints=constraints
        )
        
        optimal_weights = result.x
        metrics = self.calculate_portfolio_metrics(optimal_weights)
        metrics['weights'] = dict(zip(self.returns.columns, optimal_weights))
        
        return metrics

def calculate_returns(price_data):
    return price_data.pct_change().dropna()

In [10]:
# Example usage with price data
prices = pd.DataFrame({
    'Stock1': [100, 102, 104, 105],
    'Stock2': [45, 47, 46, 48],
    'Stock3': [82, 81, 83, 87]
}, index=['2024-01-01', '2024-01-02', '2024-01-03', '2024-01-04'])


returns = calculate_returns(prices)

optimizer = PortfolioOptimizer(returns)
optimal_portfolio = optimizer.optimize_portfolio()

print("Annual Return: {:.2%}".format(optimal_portfolio['return']))
print("Annual Risk: {:.2%}".format(optimal_portfolio['risk']))
print("Sharpe Ratio: {:.2f}".format(optimal_portfolio['sharpe_ratio']))
print("\nOptimal Portfolio Weights:")
for asset, weight in optimal_portfolio['weights'].items():
    print(f"{asset}: {weight:.2%}")

Annual Return: 437.25%
Annual Risk: 0.00%
Sharpe Ratio: 17408722.77

Optimal Portfolio Weights:
Stock1: 79.06%
Stock2: 7.22%
Stock3: 13.71%
